In [ ]:
import torch
import matplotlib.pyplot as plt
from scipy import io
import time

def delayed_matrix(X, delay):
    n, t = X.shape
    cols = t - delay + 1
    X_delayed = [X[:, i:i+cols] for i in range(delay)]
    return torch.cat(X_delayed, dim=0)

def tolerance(S, threshold=1e-6):
    S_squared = S**2
    total_energy = torch.sum(S_squared)
    cumulative = torch.cumsum(S_squared.flip(0), dim=0).flip(0)
    EE = cumulative / total_energy
    N = torch.nonzero(EE <= threshold)
    return int(N[0]) if len(N) > 0 else len(S)

def HODMD(X, delay, energy_threshold, dt):
    X_aug = delayed_matrix(X, delay)
    X1 = X_aug[:, :-1]
    X2 = X_aug[:, 1:]
    
    U, S, Vh = torch.linalg.svd(X1, full_matrices=False)
    r = tolerance(S, energy_threshold)

    Ur = U[:, :r].to(dtype=torch.complex128)
    Sr = torch.diag(S[:r]).to(dtype=torch.complex128)
    Vr = Vh.conj().T[:, :r].to(dtype=torch.complex128)
    X2_c = X2.to(dtype=torch.complex128)

    Atilde = Ur.conj().T @ X2_c @ Vr @ torch.linalg.inv(Sr)
    Lambda, W = torch.linalg.eig(Atilde)
    Phi = X2_c @ Vr @ torch.linalg.inv(Sr) @ W
    omega = torch.log(Lambda) / dt

    alpha1 = torch.linalg.lstsq(Phi, X1[:, 0].to(dtype=torch.complex128)).solution

    time_dynamics = torch.zeros((r, X1.shape[1]), dtype=torch.complex128)
    for i in range(X1.shape[1]):
        time_dynamics[:, i] = alpha1 * torch.exp(omega * ((i + 1) * dt))

    X_dmd = Phi @ time_dynamics
    return X_dmd.real.to(dtype=torch.float64), r

def main():
    vortall_mat = io.loadmat('VORTALL.mat')
    X_np = vortall_mat['VORTALL'][:, :100]
    X = torch.tensor(X_np, dtype=torch.float64)

    m, n = 199, 449
    delay = 50
    dt = 1.0
    energy_threshold = 1e-6

    start = time.time()
    X_dmd, r = HODMD(X, delay, energy_threshold, dt)
    end = time.time()

    print(f"Execution time: {end - start:.4f} seconds")
    print(f"Selected rank r = {r}")
    print("X_dmd shape:", X_dmd.shape)

    X_trimmed = X[:, delay - 1:]
    X_dmd_trimmed = X_dmd[:n * m, :]

    fig, ax = plt.subplots(2, 2, figsize=(12, 8))
    ax[0, 0].contourf(X_trimmed[:, 0].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[0, 0].set_title("Original (t=0)")
    ax[0, 0].set_aspect('equal')

    ax[0, 1].contourf(X_trimmed[:, 10].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[0, 1].set_title("Original (t=10)")
    ax[0, 1].set_aspect('equal')

    ax[1, 0].contourf(X_dmd_trimmed[:, 0].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[1, 0].set_title("HODMD Reconstructed (t=0)")
    ax[1, 0].set_aspect('equal')

    ax[1, 1].contourf(X_dmd_trimmed[:, 10].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[1, 1].set_title("HODMD Reconstructed (t=10)")
    ax[1, 1].set_aspect('equal')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

In [4]:
import torch
from tensorly.decomposition import tensor_train
import matplotlib.pyplot as plt
from scipy import io
import time
from lib import *

def delayed_matrix(X, delay):
    n, t = X.shape
    cols = t - delay + 1
    X_delayed = [X[:, i:i+cols] for i in range(delay)]
    return torch.cat(X_delayed, dim=0)

def tolerance(S, threshold=1e-6):
    S_squared = S**2
    total_energy = torch.sum(S_squared)
    cumulative = torch.cumsum(S_squared.flip(0), dim=0).flip(0)
    EE = cumulative / total_energy
    N = torch.nonzero(EE <= threshold)
    return int(N[0]) if len(N) > 0 else len(S)

def HODMD(X, delay, energy_threshold, dt):
    X_aug = delayed_matrix(X, delay)
    X1 = X_aug[..., :-1]
    X2 = X_aug[..., 1:]

    #r = [1, 10, 1]
    r = [1, 100, 1]
    X1 = np.array(X1)
    tt_cores = tensor_train(X1, rank=r)
    print(tt_cores.shape)
    return X1, r
    
    # Use double precision
    U, S, Vh = torch.linalg.svd(X1, full_matrices=False)
    r = tolerance(S, energy_threshold)

    Ur = U[:, :r].to(dtype=torch.complex128)
    Sr = torch.diag(S[:r]).to(dtype=torch.complex128)
    Vr = Vh.conj().T[:, :r].to(dtype=torch.complex128)
    X2_c = X2.to(dtype=torch.complex128)

    Atilde = Ur.conj().T @ X2_c @ Vr @ torch.linalg.inv(Sr)
    Lambda, W = torch.linalg.eig(Atilde)
    Phi = X2_c @ Vr @ torch.linalg.inv(Sr) @ W
    omega = torch.log(Lambda) / dt

    alpha1 = torch.linalg.lstsq(Phi, X1[:, 0].to(dtype=torch.complex128)).solution

    time_dynamics = torch.zeros((r, X1.shape[1]), dtype=torch.complex128)
    for i in range(X1.shape[1]):
        time_dynamics[:, i] = alpha1 * torch.exp(omega * ((i + 1) * dt))

    X_dmd = Phi @ time_dynamics
    return X_dmd.real.to(dtype=torch.float64), r

def main():
    vortall_mat = io.loadmat('VORTALL.mat')
    print(vortall_mat["VORTALL"].shape)
    X_np = vortall_mat['VORTALL'][:, :100]
    X = torch.tensor(X_np, dtype=torch.float64)
    
    m, n = 199, 449
    delay = 50
    dt = 1.0
    energy_threshold = 1e-6

    start = time.time()
    X_dmd, r = HODMD(X, delay, energy_threshold, dt)
    end = time.time()

    print(f"Execution time: {end - start:.4f} seconds")
    print(f"Selected rank r = {r}")
    print("X_dmd shape:", X_dmd.shape)

    X_trimmed = X[:, delay - 1:]
    X_dmd_trimmed = X_dmd[:n * m, :]

    fig, ax = plt.subplots(2, 2, figsize=(12, 8))
    ax[0, 0].contourf(X_trimmed[:, 0].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[0, 0].set_title("Original (t=0)")
    ax[0, 0].set_aspect('equal')

    ax[0, 1].contourf(X_trimmed[:, 10].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[0, 1].set_title("Original (t=10)")
    ax[0, 1].set_aspect('equal')

    ax[1, 0].contourf(X_dmd_trimmed[:, 0].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[1, 0].set_title("HODMD Reconstructed (t=0)")
    ax[1, 0].set_aspect('equal')

    ax[1, 1].contourf(X_dmd_trimmed[:, 10].reshape(n, m).T, levels=1001, vmin=-2, vmax=2)
    ax[1, 1].set_title("HODMD Reconstructed (t=10)")
    ax[1, 1].set_aspect('equal')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

(89351, 151)


ValueError: Provided incorrect number of ranks. Should verify len(rank) == tl.ndim(tensor)+1, but len(rank) = 7 while tl.ndim(tensor) + 1  = 3